In [3]:
import requests
import pandas as pd
import json
import getpass

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

!pip install --ignore-installed --index-url https://test.pypi.org/simple/ --no-deps sddk ### our own package under construction, always install to have up-to-date version
import sddk

Looking in indexes: https://test.pypi.org/simple/


# personal HTTP endpoint session configuration

In [17]:
### to configure HTTP endpoint (variable "sddk_url") for a personal session, leave the function parameter empty
s, sddk_url = sddk.configure_session_and_url()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
personal connection established
endpoint variable has been configured to: https://sciencedata.dk/files/


# shared folder HTTP endpoint session configuration

In [7]:
# AS FOLDER OWNER

### configure a session with endpoint in the shared folder
### (login as group owner)
s, sddk_url = sddk.configure_session_and_url("TEST_shared_folder")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
personal connection established
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/TEST_shared_folder/


In [11]:
### produce a testing dataframe:
df = pd.DataFrame([("a1", "b1", "c1"), ("a2", "b2", "c2")], columns=["a", "b", "c"]) 
df

,a,b,c
0,a1,b1,c1
1,a2,b2,c2


In [12]:
### put it to the shared folder at sciencedata 
s.put(sddk_url + "df.json", data=df.to_json())

<Response [204]>

In [13]:
### try get it back
df = pd.DataFrame(s.get(sddk_url + "df.json").json())
df

,a,b,c
0,a1,b1,c1
1,a2,b2,c2


In [14]:
# AS ORDINARY USER

### configure a session with endpoint in the shared folder
### (login as ordinary user)
s, sddk_url = sddk.configure_session_and_url("TEST_shared_folder")

sciencedata.dk username (format '123456@au.dk'): kase@zcu.cz
sciencedata.dk password: ··········
personal connection established
"TEST_shared_folder" owner's username: 648597@au.dk
connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingin/648597@au.dk/TEST_shared_folder/


In [15]:
### try to get test dataframe (produced by another user)
df = pd.DataFrame(s.get(sddk_url + "df.json").json())
df

,a,b,c
0,a1,b1,c1
1,a2,b2,c2


In [16]:
### upload there your own dataframe (as ordinary user):
s.put(sddk_url + "df_user.json", data=df.to_json())

<Response [201]>

# Functions under development... 

In [0]:
### here are the full-fledged the functions contained in the package for further development
### you can use them directly from here
### to use the package version, use "sddk.configure_session_and_url()"" 

def configure_session_and_url(shared_folder_name=None): ### insert group folder name or leave empty for personal root
  '''
  interactively setup your sciencedata.dk homeurl, username and password
  in the case of shared folders owned by someone else, ask for owner username
  '''
  sciencedata_homeurl = "https://sciencedata.dk/files/" ### your personal homeurl 
  username = input("sciencedata.dk username (format '123456@au.dk'): ")
  password = getpass.getpass("sciencedata.dk password: ")
  ### establish a request session
  s = requests.Session()
  s.auth = (username, password)
  root_folder_url = sciencedata_homeurl
  if s.get(sciencedata_homeurl).ok:
    print("personal connection established")
    if shared_folder_name != None:
      ### SETTING FOR SHARED FOLDER (you either have to be its owner, or at least know username of its owner)
      shared_folder_owner_url = "https://sciencedata.dk/files/" + shared_folder_name + "/"
      if s.get(shared_folder_owner_url).ok: ### if you are owner of the shared folder 
        root_folder_url = shared_folder_owner_url ### use the url as the endpoint
        print("connection with shared folder established with you as its owner")
      else: # otherwise use endpoint for "shared with me"
        folder_owner = input("\"" + shared_folder_name + "\" owner's username: ") ### in the case Vojtech is folder owner
        shared_folder_member_url = "https://sciencedata.dk/sharingin/" + folder_owner + "/" + shared_folder_name + "/" 
        if s.get(shared_folder_member_url).ok:
          root_folder_url = shared_folder_member_url
          print("connection with shared folder established with you as its ordinary user")
        else:
          print("connection with shared folder failed")
  print("endpoint variable has been configured to: " + root_folder_url)
  return s, root_folder_url

def file_from_object(file_name_and_loc, python_object):
  if s.get(sciencedata_groupurl + file_name_and_loc).ok: ### if there already is a file with the same name
    new_name = input("file with name \"" + file_name_and_loc.rpartition("/")[2] + "\" already exists in given location. Press Enter to overwrite it or enter a different name (without path)")
    if len(new_name) == 0:
      s.put(sciencedata_groupurl + file_name_and_loc, data=json.dumps(python_object))
    else:
      if "/" in new_name: ### if it is a path
        s.put(sciencedata_groupurl + new_name, data=json.dumps(python_object))
      else: 
        s.put(sciencedata_groupurl + file_name_and_loc.rpartition("/")[0] + new_name, data=json.dumps(python_object))
  else:
    s.put(sciencedata_groupurl + file_name_and_loc, data=json.dumps(python_object))

def object_from_file(file_name_and_loc):
  if s.get(sciencedata_groupurl + file_name_and_loc).ok:
    print("file exists")
    try: 
      return json.loads(s.get(sciencedata_groupurl + file_name_and_loc).content) ### if there already is a file with the same name
    except:
      print("file import failed")
  else:
    print("file does not found; check file name and path.")


In [0]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

In [0]:
file_from_object(sciencedata_url + "textfile.txt", "string to write into file")

In [0]:
edh_xml_data_df = pd.DataFrame(s.get(sciencedata_groupurl + "SDAM_data/EDH/edh_xml_data_df.json").json())

In [0]:
edh_xml_data_df.head(5)

,idno_uri,idno_tm,placenames_refs,text,origdate_text,objecttype,commentary
0,http://edh-www.adw.uni-heidelberg.de/edh/insch...,214091,"[http://www.trismegistos.org/place/019860, htt...",In hoc tumulo meserecordia! Christi requiescet...,551 AD – 600 AD,"[Tafel, 257]",\n\n
1,http://edh-www.adw.uni-heidelberg.de/edh/insch...,415668,"[http://www.trismegistos.org/place/016530, htt...",Alae Noricorum Iulius Quintus Vagdaevercusti v...,71 AD – 130 AD,"[Tafel, 257]",\n\n
2,http://edh-www.adw.uni-heidelberg.de/edh/insch...,192497,"[http://www.trismegistos.org/place/029481, htt...",Imperator Caesar Lucius Septimius Severus Pius...,197 AD,"[unbestimmt, 2]",\n Textwiedergabe nach Eck.\n
3,http://edh-www.adw.uni-heidelberg.de/edh/insch...,199108,"[http://www.trismegistos.org/place/015771, htt...",Saturno Meddensi Augusto sacru...,,"[unbestimmt, 2]",\n\n
4,http://edh-www.adw.uni-heidelberg.de/edh/insch...,201034,"[http://www.trismegistos.org/place/015771, htt...",Lucio Naevio Quadratiano legato Augusti pro pr...,193 AD,"[unbestimmt, 2]",\n Datierung: L. Naevius Quadratianus war 193 ...
